<a href="https://colab.research.google.com/github/cytoscape/cytoscape-automation/blob/master/for-scripters/Python/custom-graphics-and-labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Graphics and Labels


## Yihang Xin and Alex Pico

## 2020-12-15

This vignette illustrates how Cytoscape’s Custom Graphics can be used to add graphs, charts and other graphics to node, and how to combine Custom Graphics with the enhancedGraphics app for specialized visualizations.



# Installation
The following chunk of code installs the `py4cytoscape` module.

In [ ]:
%%capture
!python3 -m pip install python-igraph requests pandas networkx
!python3 -m pip install py4cytoscape

If you are using a remote notebook environment such as Google Colab, please execute the cell below. (If you're running on your local notebook, you don't need to do that.)



In [ ]:
import requests
exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)
IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

# Prerequisites


In addition to this package (py4cytoscape version 0.0.7), you will need:

* Install the filetransfer app from https://apps.cytoscape.org/apps/filetransfer

* Latest version of Cytoscape, which can be downloaded from https://cytoscape.org/download.html. Simply follow the installation instructions on screen.

* Complete installation wizard

* Launch Cytoscape

You can also install app inside Python notebook by running "py4cytoscape.install_app('Your App')"

# Import the required package


In [ ]:
import os
import sys
import pandas as pd
import py4cytoscape as p4c

# Setup Cytoscape


In [ ]:
p4c.cytoscape_version_info()

{'apiVersion': 'v1',
 'cytoscapeVersion': '3.8.2',
 'automationAPIVersion': '1.0.0',
 'py4cytoscapeVersion': '0.0.6'}

# Open Sample

For this tutorial, we will be using the galFiltered sample session file, which includes a yeast network and associated data.

In [ ]:
p4c.open_session()

Opening sampleData/sessions/Yeast Perturbation.cys...


{}

# Set style and node color


First, lets change the style to a simple default and the color of nodes to grey:

In [ ]:
p4c.set_visual_style('default')
p4c.set_node_color_default('#D8D8D8')

style_name not specified; updating "default" style.


''

# Custom Graphics

## Bar chart

In this example, we will create a bar chart with the three expression values, gal1RGexp, gal4RGexp and gal80Rexp, available as attributes in the session file.

Create the Custom Graphic:

In [ ]:
p4c.set_node_custom_bar_chart(["gal1RGexp","gal4RGexp","gal80Rexp"])

style_name not specified; updating "default" style.


''

There are 4 types of Bar Charts and customizable parameters for colors, size, spacing and orientation.

Position the Bar Chart just below the node. This is an optional step that we are doing here just to make room for subsequent graphics. By specifying both anchors at opposite ends, we can get a lot of space between the node and the graphic.

In [ ]:
p4c.set_node_custom_position(node_anchor='S', graphic_anchor='N',justification='c')

style_name not specified; updating "default" style.


''

# Stripes

Next we are going to create stripes of gradient mappings using a horizontal “heatmap”" of the same three data columns and position the heatmap right above the node. For this vignette, we need to also specify the slot number to avoid overwriting the Bar Chart:

In [ ]:
p4c.set_node_custom_heat_map_chart(["gal1RGexp","gal4RGexp","gal80Rexp"], slot = 2)
p4c.set_node_custom_position(node_anchor='N', graphic_anchor='S', justification='c', slot = 2)

style_name not specified; updating "default" style.
style_name not specified; updating "default" style.


''

# Pie chart

Finally, we will create a pie chart with two columns, Radiality and Degree, and place it to the left of the node. Here we’ll use the xOffset parameter to be even more specific about where we want to place the graphic relative to the node.

In [ ]:
p4c.set_node_custom_pie_chart(['Radiality', 'Degree'], slot = 3)
p4c.set_node_custom_position(node_anchor='W', justification='c', x_offset = -20, slot = 3)

style_name not specified; updating "default" style.
style_name not specified; updating "default" style.


''

# Enhanced Graphics


The nodes in the network are labeled with the corresponding protein names (yeast), but there is additional text information in the Node Table that could be useful to display as labels on the nodes. We are going to use the enhancedGraphics app to create a second node label for the common yeast gene name.

This involves a new step: Filling a new column with parameters for the enhancedGraphics App. This column is then mapped to a Custom Graphic slot and (optionally) positioned, like in the examples above.



# Install enhancedGraphics


The enhancedGraphics app is available from the Cytoscape App Store.

Install the enhancedGraphics app from http://apps.cytoscape.org/apps/enhancedgraphics

You can also install app inside Python notebook by running py4cytoscape.install_app('Your App')

# Define new label


The new column values have to follow a specific syntax to be recognized by the enhancedGraphics app. Here, for example, is how you set a label based on another attribute (e.g., the column called “COMMON”), specifying its size, color, outline and background:

"label: attribute=COMMON labelsize=10 color=red outline=false background=false""


For more details on the enhancedGraphics format, see the manual https://www.cgl.ucsf.edu/cytoscape/utilities3/enhancedcg.shtml.

First, we define a dataframe with two columns: node names (“name”) and the new label (“my second label”):



In [ ]:
all_nodes = p4c.get_all_nodes()

In [ ]:
label_df = pd.DataFrame(data=all_nodes, columns=['name'])

In [ ]:
label_df["my second label"] = "label: attribute=COMMON labelsize=10 color=red outline=false background=false"

Next, we load this dataframe into the Node Table to create and fill a new column:



In [ ]:
p4c.load_table_data(label_df, data_key_column='name', table_key_column='name')

'Success: Data loaded in defaultnode table'

# Map and position label


We now have a new column, my second label, that we can use for the mapping. This mapping does not come with a custom helper function, se we are going to use two alternative functions to prepare the passthrough mapping property and then update our visual style with the new mapping:



In [ ]:
label_map = p4c.map_visual_property('node customgraphics 4', 'my second label', 'p')
p4c.update_style_mapping('default', label_map)

''

Note: the custom graphic slot number is actulally part of the property’s name.

Finally, we position the new label in the upper right corner:

In [ ]:
p4c.set_node_custom_position(node_anchor='E', graphic_anchor='C', justification='r' , x_offset = 40,  slot = 4)

style_name not specified; updating "default" style.


''